In [171]:
import pandas as pd
import collections
import os
# import nltk

In [172]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 
punc = set([',', '-', '/', '.', '  ', 'university', 'research', 'center', 'institute'])
stop_words.update(punc)
'of' in stop_words

True

In [173]:
from fuzzywuzzy import fuzz

key = 'University of Illinois Urbana - Champaign'
value = 'University of Illinois at Urbana, Champaign'

def clean(name):
    name = name.lower()
    clean_name = []
    tok = word_tokenize(name)
    for token in tok:
        if token in stop_words:
            continue
        else:
            clean_name.append(token)
    return " ".join(clean_name)


def fuzzy(key, value):
    key = clean(key)
    value = clean(value)
    fuzz_score = fuzz.token_set_ratio(key, value)
    return fuzz_score

fuzzy('University of British Columbia', 'U British Columbia')

100

In [174]:
#creating a csv file to contain all the conferences TPC details

ipsn_tpc = pd.read_csv('data/Clean_TPC/ipsn_clean.csv')
ipsn_auth = pd.read_csv('data/Clean_Authors/ipsn_authors.csv')

mobicom_tpc = pd.read_csv('data/Clean_TPC/mobicom_clean.csv')
mobicom_auth = pd.read_csv('data/Clean_Authors/mobicom_authors.csv')

mobihoc_tpc = pd.read_csv('data/Clean_TPC/mobihoc_clean.csv')
mobihoc_auth = pd.read_csv('data/Clean_Authors/mobihoc_authors.csv')

sensys_tpc = pd.read_csv('data/Clean_TPC/sensys_clean.csv')
sensys_auth = pd.read_csv('data/Clean_Authors/sensys_authors.csv')

sigcomm_tpc = pd.read_csv('data/Clean_TPC/sigcomm_clean.csv')
sigcomm_auth = pd.read_csv('data/Clean_Authors/sigcomm_authors.csv')

In [204]:
def create_tup_list(author_df, tpc_df):
    "Given the author df for a given conf, returns the tup list of author and uni"
    
    tpc_uni = set(tpc_df['University/Organization'])
    auth_uni = set(author_df['University/Organization'])
    tup_list = []
    
    for t_uni in tpc_uni:
        for a_uni in auth_uni:
            try:
                fuzz_score = fuzzy(t_uni, a_uni)
                if fuzz_score > 95:
                    tup_list.append((t_uni, a_uni))
            except:
                continue
    return tup_list
create_tup_list(ipsn_auth, ipsn_tpc)

[('University of Houston, USA', 'University of Houston'),
 ('Crossbow', 'Crossbow Technology'),
 ('Intel Labs', 'Intel'),
 ('Intel Labs', 'Intel Research'),
 ('Institute of Computing Technology, Chinese Academy of Sciences',
  'Institute of Computing Technology'),
 ('Institute of Computing Technology, Chinese Academy of Sciences',
  'Chinese Academy of Sciences and University of Chinese Academy of Sciences'),
 ('Institute of Computing Technology, Chinese Academy of Sciences',
  'Chinese Academy of Sciences'),
 ('KTH Royal Institute of Technology', 'KTH Royal Institute of Technology'),
 ('KTH Royal Institute of Technology', 'Royal Institute of Technology'),
 ('Maryland', 'University of Maryland College Park'),
 ('Maryland', 'University of Maryland'),
 ('University of California San Diego', 'University of California San Diego'),
 ('University of California San Diego', 'University of California'),
 ('Swedish Institute of Computer Science, Sweden',
  'Swedish Institute of Computer Science'

In [206]:
def auth_tpc_df(author_df, tpc_df):
    tup_list = create_tup_list(author_df, tpc_df)
    "Given the tup list creates two dataframes one from auth info and other from tpc info"
    author_name = []
    publications = []
    auth_univ = []

    tpc_univ = []
    tpc_name = []

    for tup in tup_list:

        auth_uni, tpc_uni = tup

        #extract author name and publication for auth_uni
        auth_df = auth_df[auth_df['University/Organization'] == auth_uni]
        author_name.extend(list(auth_df['Author_Name']))
        publications.extend(list(auth_df['Paper_title']))

        tpc_df = tpd_df[tpc_df['University/Organization'] == tpc_uni]
        tpc_name.extend(list(tpc_df['Name']))

        if len(auth_uni) < len(tpc_uni):
            auth_univ.extend([auth_uni])
            tpc_univ.extend([auth_uni])
        else:
            auth_univ.extend([tpc_uni])
            tpc_univ.extend([tpc_uni])

    a = pd.DataFrame(list(zip(author_name, auth_univ, publications)), columns = ['Author_Name', 'University/Organization', 'Paper'])
    b = pd.DataFrame(list(zip(tpc_name, tpc_univ)), columns = ['TPC_Name', 'University/Organization'])
    
    return a, b

auth_tpc_df(ipsn_auth, ipsn_tpc)

(                 Author_Name            University/Organization  \
 0             António  Gonga              University of Houston   
 1             Pablo  Soldati                           Crossbow   
 2          Mikael  Johansson                              Intel   
 3           Euhanna  Ghadimi                         Intel Labs   
 4          Mikael  Johansson  Institute of Computing Technology   
 ..                       ...                                ...   
 344            Jianxun  Wang                     Intel Research   
 345              Xiaobo  Tan                  Temple University   
 346  Mohammad Mahdi Moazzami             Technische Universität   
 347       Dennis E. Phillips                            HP Labs   
 348           Guoliang  Xing             RWTH Aachen University   
 
                                                  Paper  
 0    Multi-Channel Communication vs. Adaptive Routi...  
 1    Multi-Channel Communication vs. Adaptive Routi...  
 2    Mu

In [210]:
def bubble_df(a, b):
    "Given the dataframes extracted as output return the final dataframe needed for bubble chart"
    
    uni_auth_count = collections.defaultdict(int)
    uni_paper_count = collections.defaultdict(int)
    tot = 0

    for index in range(len(a)):
        uni = a['University/Organization'][index]
        auth_name = a['Author_Name'][index]
        paper = a['Paper'][index]

        if uni not in uni_auth_count.keys():
            uni_auth_count[uni] = 1

        elif uni in uni_auth_count.keys():
            uni_auth_count[uni] += 1

        if uni not in uni_paper_count.keys():
            uni_paper_count[uni] = 1

        elif uni in uni_paper_count.keys():
            uni_paper_count[uni] += 1
            
            
    uni_tpc_count = collections.defaultdict(int)
    for index in range(len(b)):
        uni = b['University/Organization'][index]
        tpc = b['TPC_Name'][index]

        if uni not in uni_tpc_count.keys():
            uni_tpc_count[uni] = 1
        elif uni in uni_tpc_count.keys():
            uni_tpc_count[uni] += 1
            
    no_auth_df = pd.DataFrame(list(zip(list(uni_tpc_count.keys()), list(uni_tpc_count.values()))), columns = ['University/Organization', 'No_of_TPC'])
    no_tpc_df = pd.DataFrame(list(zip(list(uni_auth_count.keys()), list(uni_auth_count.values()))), columns = ['University/Organization', 'No_of_authors'])

    bubble_df = pd.merge(left = no_auth_df, right = no_tpc_df, left_on='University/Organization', right_on='University/Organization')
    
    return bubble_df

In [211]:
a, b = auth_tpc_df(ipsn_auth, ipsn_tpc)
final_df = bubble_df(a, b)

In [215]:
len(final_df)

166

In [214]:
final_df[final_df['No_of_TPC'] == final_df['No_of_authors']]

,University/Organization,No_of_TPC,No_of_authors
0,University of Houston,3,3
1,Crossbow,1,1
2,Intel,9,9
3,Intel Labs,1,1
4,Institute of Computing Technology,1,1
...,...,...,...
161,Intel Research,5,5
162,Temple University,1,1
163,Technische Universität,1,1
164,HP Labs,1,1
